In [1]:
from Datasets.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [3]:
# Load and prepare data
def load_prep_data(dataset_name):
    if dataset_name in ['Adult', 'Bank']:
        dataset = Dataset(dataset_name)
        X, y, feature_names, x = dataset.load_data()
        X_old, y_old, X_new, y_new = dataset.split_old_new_data(X, y)
        return X_old, y_old, X_new, y_new, feature_names, x
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")

In [4]:
X_old, y_old, X_new, y_new, feature_names, x = load_prep_data('Adult')
# Verify the split
print(f"Old data shape: {X_old}, Positive samples: {y_old.sum()}")
print(f"New data shape: {X_new}, Positive samples: {y_new.sum()}")

Old data shape: [[2.60000e+01 2.22248e+05 9.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [4.90000e+01 1.33729e+05 1.00000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [5.50000e+01 1.02058e+05 6.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 ...
 [4.40000e+01 2.48973e+05 1.30000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [2.20000e+01 1.48187e+05 7.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [3.50000e+01 3.10950e+04 1.00000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]], Positive samples: 3536
New data shape: [[2.80000e+01 3.38409e+05 1.30000e+01 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [5.20000e+01 2.09642e+05 9.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [3.00000e+01 1.41297e+05 1.30000e+01 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 ...
 [2.70000e+01 2.57302e+05 1.20000e+01 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [4.00000e+01 1.54374e+05 9.00000e+00 ... 1.00000e+00 0.00000e+00
  0.00000e+00]
 [5.20000e+01 2.87927e+05 9.00000e+00 ... 

In [5]:
from Models.models import train_old_model, train_new_models
from Models.lct import LocalCorrectionTree

/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
'''
###############################################################################
# Test Function
###############################################################################
def test_local_correction_tree():
    """
    Tests LocalCorrectionTree using random data.
    Includes assertions for each main step: fit, predict, prune, simplify.
    """
    from numpy.random import rand, randint, randn, seed

    seed(42)
    n_samples = 60
    n_features = 5
    n_classes = 3
    X = rand(n_samples, n_features)
    y = randint(0, n_classes, size=n_samples)
    old_scores = randn(n_samples, n_classes)

    # Instantiate LCT
    lct = LocalCorrectionTree(lambda_reg=0.1, max_depth=3, min_samples_leaf=5)

    lct.fit(X, y, old_scores)
    corrections = lct.predict(X)
    assert corrections.shape == (n_samples, n_classes), "Predict output shape is incorrect"

    pre_prune_nodes = len(lct.nodes)
    lct.prune(X, y, old_scores, nprune=2)
    post_prune_nodes = len(lct.nodes)
    assert post_prune_nodes <= pre_prune_nodes, "Prune should not increase node count"

    pre_simplify_nodes = len(lct.nodes)
    lct.simplify()
    post_simplify_nodes = len(lct.nodes)
    assert post_simplify_nodes <= pre_simplify_nodes, "Simplify should not increase node count"

    print("All tests for LocalCorrectionTree passed successfully!")

###############################################################################
# Example: Randomly Sample "lambda_reg" with Optuna
###############################################################################
def objective(trial):
    """
    Optuna objective function that samples lambda_reg in [0.05, 0.5],
    then trains an LCT on random data and returns a dummy 'loss'.
    Replace this dummy training code with real data if you want a meaningful search.
    """
    # Sample random lambda
    lambda_reg = trial.suggest_float("lambda_reg", 0.05, 0.5, log=False)

    np.random.seed(42)
    X = np.random.rand(50, 5)
    y = np.random.randint(0, 3, size=50)
    old_scores = np.random.randn(50, 3)

    lct_temp = LocalCorrectionTree(
        lambda_reg=lambda_reg,
        max_depth=3,
        min_samples_leaf=5,
    )
    # "Train" on the random data
    lct_temp.fit(X, y, old_scores)

    # Evaluate some dummy metric (for demonstration).
    # Typically, you'd do cross-validation or measure real performance here.
    preds = lct_temp.predict(X)
    # We'll create a dummy "loss" = sum of squares of corrections, smaller is "better"
    dummy_loss = float(np.sum(preds**2))
    return dummy_loss

#if __name__ == "__main__":
    # 1. Run your LCT tests
    #test_local_correction_tree()

    # 2. Optimizing with real random sampling for lambda_reg
    #import optuna
    #study = optuna.create_study(direction="minimize")
    #study.optimize(objective, n_trials=10)  # e.g. 10 random trials

    # 3. Print best trial and use it to instantiate LCT
    #best_trial = study.best_trial
    #best_lambda = best_trial.params["lambda_reg"]
    #print(f"\nOptuna found best lambda_reg={best_lambda:.3f} with objective={best_trial.value:.3f}.")

    ## 4. Example: build a final LCT with the best lambda and possibly real data
    #final_lct = LocalCorrectionTree(lambda_reg=best_lambda, max_depth=3, min_samples_leaf=5)
    #print(f"Final LCT instantiated with lambda_reg={best_lambda:.3f}.")'''


'\n###############################################################################\n# Test Function\n###############################################################################\ndef test_local_correction_tree():\n    """\n    Tests LocalCorrectionTree using random data.\n    Includes assertions for each main step: fit, predict, prune, simplify.\n    """\n    from numpy.random import rand, randint, randn, seed\n\n    seed(42)\n    n_samples = 60\n    n_features = 5\n    n_classes = 3\n    X = rand(n_samples, n_features)\n    y = randint(0, n_classes, size=n_samples)\n    old_scores = randn(n_samples, n_classes)\n\n    # Instantiate LCT\n    lct = LocalCorrectionTree(lambda_reg=0.1, max_depth=3, min_samples_leaf=5)\n\n    lct.fit(X, y, old_scores)\n    corrections = lct.predict(X)\n    assert corrections.shape == (n_samples, n_classes), "Predict output shape is incorrect"\n\n    pre_prune_nodes = len(lct.nodes)\n    lct.prune(X, y, old_scores, nprune=2)\n    post_prune_nodes = len(lc

In [7]:
'''import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

dataset = Dataset("Adult")
X, y, feature_names, raw_data = dataset.load_data()
X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple logistic regression model as the "old model"
old_model = LogisticRegression(random_state=42)
old_model.fit(X_train, y_train)

# Get old model scores on the test set
old_scores = old_model.predict_proba(X_test)

# Instantiate and fit the Local Correction Tree
lct = LocalCorrectionTree()
lct.fit(X_test, y_test, old_scores)

# Get corrections from LCT
corrections = lct.predict(X_test)

# Apply corrections to old model scores
corrected_scores = old_scores + corrections

# Evaluate the performance
old_accuracy = np.mean(np.argmax(old_scores, axis=1) == y_test)
new_accuracy = np.mean(np.argmax(corrected_scores, axis=1) == y_test)

print(f"Old model accuracy: {old_accuracy:.4f}")
print(f"Corrected model accuracy: {new_accuracy:.4f}")

# Analyze the corrections
non_zero_corrections = np.sum(np.any(corrections != 0, axis=1))
print(f"Number of samples with non-zero corrections: {non_zero_corrections}")
print(f"Percentage of samples corrected: {non_zero_corrections / len(X_test) * 100:.2f}%")

# Print a few example corrections along with labels before and after correction
print("\nExample corrections:")
for i in range(10):
    old_label = np.argmax(old_scores[i])
    corrected_label = np.argmax(corrected_scores[i])
    print(f"Sample {i}:")
    print(f"  Old scores: {old_scores[i]}")
    print(f"  Correction: {corrections[i]}")
    print(f"  New scores: {corrected_scores[i]}")
    print(f"  True label: {y_test[i]}")
    print(f"  Predicted label before correction: {old_label}")
    print(f"  Predicted label after correction: {corrected_label}")
    print()

# Visualize the tree structure and include labels before and after correction
lct.simplify()
print("\nSimplified tree structure:")
def print_tree(node_id, depth=0):
    feature_idx, threshold, w_node = lct.nodes[node_id]
    left_id, right_id = lct.children[node_id]
    
    if feature_idx == -1:
        print("  " * depth + f"Leaf: correction = {w_node}")
    else:
        feature_name = feature_names[feature_idx]
        print("  " * depth + f"Node: {feature_name} <= {threshold:.4f}")
        print_tree(left_id, depth + 1)
        print_tree(right_id, depth + 1)

print_tree(0) '''

'import numpy as np\nfrom sklearn.datasets import make_classification\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LogisticRegression\n\ndataset = Dataset("Adult")\nX, y, feature_names, raw_data = dataset.load_data()\nX, y = np.array(X), np.array(y)\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Train a simple logistic regression model as the "old model"\nold_model = LogisticRegression(random_state=42)\nold_model.fit(X_train, y_train)\n\n# Get old model scores on the test set\nold_scores = old_model.predict_proba(X_test)\n\n# Instantiate and fit the Local Correction Tree\nlct = LocalCorrectionTree()\nlct.fit(X_test, y_test, old_scores)\n\n# Get corrections from LCT\ncorrections = lct.predict(X_test)\n\n# Apply corrections to old model scores\ncorrected_scores = old_scores + corrections\n\n# Evaluate the performance\nold_accuracy = np.mean(np.argmax(old_scores, axis=1) == y_test)\nnew_accuracy 

In [8]:
'''def _find_w(self, indices, old_scores, y, max_iter=10):
        #n_samples = len(indices)
        n_classes = old_scores.shape[1]
        
        w = np.zeros((1, n_classes))
        g_avg = np.zeros((1, n_classes))
        beta_t = 1.0  # Example sequence, adjust as needed
        
        for t in range(1, max_iter + 1):
            for i in indices:
                # Compute subgradient of the loss function for sample i
                sub_scores = old_scores[i] + w
                probs = np.exp(sub_scores) / np.sum(np.exp(sub_scores))
                subgradient = -np.eye(n_classes)[y[i]] + probs
                
                # Update average subgradient
                g_avg = (t - 1) / t * g_avg + 1 / t * subgradient
                
                # Update w using RDA formula
                w = w - beta_t / t * g_avg - self.lambda_reg * w
        
        return w'''

'def _find_w(self, indices, old_scores, y, max_iter=10):\n        #n_samples = len(indices)\n        n_classes = old_scores.shape[1]\n        \n        w = np.zeros((1, n_classes))\n        g_avg = np.zeros((1, n_classes))\n        beta_t = 1.0  # Example sequence, adjust as needed\n        \n        for t in range(1, max_iter + 1):\n            for i in indices:\n                # Compute subgradient of the loss function for sample i\n                sub_scores = old_scores[i] + w\n                probs = np.exp(sub_scores) / np.sum(np.exp(sub_scores))\n                subgradient = -np.eye(n_classes)[y[i]] + probs\n                \n                # Update average subgradient\n                g_avg = (t - 1) / t * g_avg + 1 / t * subgradient\n                \n                # Update w using RDA formula\n                w = w - beta_t / t * g_avg - self.lambda_reg * w\n        \n        return w'

In [9]:
def evaluate_old_model(dataset, X, y, old_model):
    """
    Evaluate the old model on the given dataset (X, y).
    Returns a dictionary of metrics with lists of values.
    """
    y_pred = old_model.predict(X)
    is_multiclass = dataset in ['CTG', '7-point']
    
    if is_multiclass:
        precision = precision_score(y, y_pred, average='macro', zero_division=0)
        recall = recall_score(y, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y, y_pred, average='macro', zero_division=0)
    else:
        precision = precision_score(y, y_pred, zero_division=0)
        recall = recall_score(y, y_pred, zero_division=0)
        f1 = f1_score(y, y_pred, zero_division=0)
    
    accuracy = accuracy_score(y, y_pred)
    
    return {
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1': [f1]
    }

In [10]:
def evaluate_new_models(dataset, models, X_test, y_test, base_models_dict):
    """
    Evaluate new models including LCT on the test set.
    Returns metrics formatted for error bar plotting.
    """
    metrics = {
        'Accuracy': {},
        'Precision': {},
        'Recall': {},
        'F1': {}
    }
    
    is_multiclass = dataset in ['CTG', '7-point']
    
    for model_name, model in models.items():
        base_model_name = model_name.replace('+', '').replace('C', '')
        old_scores_test = base_models_dict[base_model_name].predict_proba(X_test)
        
        if model_name == 'Ours':  # LCT model
            corrections = model.predict(X_test)
            corrected_scores = old_scores_test + corrections
            y_pred = np.argmax(corrected_scores, axis=1)
        elif model_name.endswith('+') or model_name.endswith('+C'):
            # Models with old scores as additional features
            X_test_with_scores = np.hstack([X_test, old_scores_test])
            y_pred = model.predict(X_test_with_scores)
        else:
            # Base models
            y_pred = model.predict(X_test)
        
        if is_multiclass:
            precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
            recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
        else:
            precision = precision_score(y_test, y_pred, zero_division=0)
            recall = recall_score(y_test, y_pred, zero_division=0)
            f1 = f1_score(y_test, y_pred, zero_division=0)
        
        accuracy = accuracy_score(y_test, y_pred)
        
        metrics['Accuracy'][model_name] = accuracy
        metrics['Precision'][model_name] = precision
        metrics['Recall'][model_name] = recall
        metrics['F1'][model_name] = f1
    
    return metrics


In [11]:
def plot_metrics_comparison(dataset, metrics, model_names):
    """
    Create comparison bar plots for model metrics.
    """
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1']
    
    # Define model colors and markers
    colors = {
        'L1-LR': '#1f77b4',
        'L1-LR+': '#1f77b4',
        'L2-LR': '#ff7f0e',
        'L2-LR+': '#ff7f0e',
        'DT': '#2ca02c',
        'DT+': '#2ca02c',
        'RF': '#8c564b',
        'RF+': '#8c564b',
        'LGBM': '#9467bd',
        'LGBM+': '#9467bd',
        'LGBM+C': '#9467bd',
        'Ours': '#d62728',
        'Old': '#7f7f7f'
    }
    
    for idx, metric in enumerate(metrics_names):
        ax = axes[idx]
        y_pos = np.arange(len(model_names))
        values = metrics[metric]
        
        # Plot bar plots
        for i, (name, value) in enumerate(zip(model_names, values)):
            color = colors.get(name)
            mean_val = np.mean(value)
            std_val = np.std(value)
            
            # Plot bar
            ax.bar(i, mean_val, color=color, alpha=0.7, width=0.8)
            
            # Plot error bars
            ax.errorbar(i, mean_val, yerr=std_val, fmt='none', capsize=3, color='black', elinewidth=1, capthick=1)
        
        ax.set_yticks(np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 5))
        ax.set_xticks(y_pos)
        ax.set_xticklabels(model_names, rotation=90)
        ax.set_title(f'({chr(97+idx)}) {metric} ({dataset})')
        ax.grid(True, alpha=0.3, linestyle='--')
        
        # Set y-axis limits
        min_val = min([np.mean(v) for v in values]) - 0.05
        max_val = max([np.mean(v) for v in values]) + 0.05
        ax.set_ylim(min_val, max_val)
    
    plt.tight_layout()
    return fig

In [12]:
def plot_evaluation_metrics(metrics, models, dataset, color_scheme):
    """Create subplots for all metrics with different colors for each model"""
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1']
            
    for idx, (ax, metric) in enumerate(zip(axes, metric_names)):
        means = []
        stds = []
                
        for model in models:
            mean_val = np.mean(metrics[metric][model])
            std_val = np.std(metrics[metric][model])
            means.append(mean_val)
            stds.append(std_val)
                    
            # Create horizontal error bar plot with different colors
            y_pos = np.arange(len(models))
            ax.errorbar(mean_val, models.index(model), xerr=std_val, 
                        fmt='o', capsize=5, color=color_scheme[model],
                        label=model, markersize=8, elinewidth=2, capthick=2)
                
            # Customize subplot
            ax.set_yticks(y_pos)
            ax.set_yticklabels(models)
            ax.set_xlabel(metric)
            ax.set_title(f'({chr(97+idx)}) {metric} ({dataset})')
            ax.grid(True, alpha=0.3)
                
            # Set x-axis limits with margin
            margin = 0.02
            ax.set_xlim(min(means) - margin, max(means) + margin)
            
        plt.tight_layout()
        return fig

In [13]:
def print_tree(lct, node_id=0, depth=0, feature_names=None):
    """
    Print the Local Correction Tree structure.
    Args:
        lct: LocalCorrectionTree instance
        node_id: Current node ID
        depth: Current depth in tree
        feature_names: Optional list of feature names
    """
    feature_idx, threshold, w_node = lct.nodes[node_id]
    left_id, right_id = lct.children[node_id]
    
    indent = "  " * depth
    if feature_idx == -1:
        print(f"{indent}Leaf: correction = {w_node.round(4)}")
    else:
        feature = f"X[{feature_idx}]" if feature_names is None else feature_names[feature_idx]
        print(f"{indent}Node: {feature} <= {threshold:.4f}")
        print_tree(lct, left_id, depth + 1, feature_names)
        print_tree(lct, right_id, depth + 1, feature_names)

In [14]:
'''datasets = ['Adult', 'Bank']
for dataset in datasets:
    print(f"\nProcessing {dataset} dataset")
    
    # Define only base models
    base_models = ['L1-LR', 'L2-LR', 'DT', 'RF', 'LGBM']
    
    # 1. Load data
    if dataset == '7-point':
        X_old, y_old, X_new, y_new, X_clinical = load_prep_data(dataset)
    else:
        X_old, y_old, X_new, y_new, feature_names, x = load_prep_data(dataset)
    
    X_new, X_old = np.array(X_new), np.array(X_old)
    y_new, y_old = np.array(y_new), np.array(y_old)
    
    # 2. Train base models on old data
    base_models_dict = train_old_model(dataset, X_new, X_old, y_new, y_old)
    
    # 3. Perform 5-fold cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_metrics = {
        'Accuracy': {model: [] for model in base_models},
        'Precision': {model: [] for model in base_models},
        'Recall': {model: [] for model in base_models},
        'F1': {model: [] for model in base_models}
    }
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_old, y_old)):
        print(f"Processing fold {fold + 1}/5")
        
        X_train, X_test = X_old[train_idx], X_old[test_idx]
        y_train, y_test = y_old[train_idx], y_old[test_idx]
        
        # Evaluate each base model individually
        for model_name in base_models:
            fold_metrics = evaluate_old_model(dataset, X_test, y_test, base_models_dict[model_name])
            
            for metric in all_metrics:
                all_metrics[metric][model_name].extend(fold_metrics[metric])
    
    # 4. Prepare metrics for plotting
    plot_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    for metric in plot_metrics:
        for model in base_models:
            plot_metrics[metric].append(all_metrics[metric][model])
    
    # 5. Create and save plot
    fig = plot_metrics_comparison(dataset, plot_metrics, base_models)
    plt.savefig(f'{dataset}_base_models_comparison.png', bbox_inches='tight', dpi=400)
    plt.close()
    
    # 6. Print numerical results
    print(f"\nBase Model Results for {dataset}:")
    for metric in plot_metrics:
        print(f"\n{metric}:")
        for name, values in zip(base_models, plot_metrics[metric]):
            mean_val = np.mean(values)
            std_val = np.std(values)
            print(f"{name}: {mean_val:.4f} ± {std_val:.4f}")

print("\nDone evaluating base models.")'''

'datasets = [\'Adult\', \'Bank\']\nfor dataset in datasets:\n    print(f"\nProcessing {dataset} dataset")\n    \n    # Define only base models\n    base_models = [\'L1-LR\', \'L2-LR\', \'DT\', \'RF\', \'LGBM\']\n    \n    # 1. Load data\n    if dataset == \'7-point\':\n        X_old, y_old, X_new, y_new, X_clinical = load_prep_data(dataset)\n    else:\n        X_old, y_old, X_new, y_new, feature_names, x = load_prep_data(dataset)\n    \n    X_new, X_old = np.array(X_new), np.array(X_old)\n    y_new, y_old = np.array(y_new), np.array(y_old)\n    \n    # 2. Train base models on old data\n    base_models_dict = train_old_model(dataset, X_new, X_old, y_new, y_old)\n    \n    # 3. Perform 5-fold cross validation\n    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n    all_metrics = {\n        \'Accuracy\': {model: [] for model in base_models},\n        \'Precision\': {model: [] for model in base_models},\n        \'Recall\': {model: [] for model in base_models},\n        \

In [ ]:
datasets = ['Adult', 'Bank']
for dataset in datasets:
    print(f"\nProcessing {dataset} dataset")
    
    # 1. Load data
    if dataset == '7-point':
        X_old, y_old, X_new, y_new, X_clinical = load_prep_data(dataset)
    else:
        X_old, y_old, X_new, y_new, feature_names, x = load_prep_data(dataset)
    
    X_new, X_old = np.array(X_new), np.array(X_old)
    y_new, y_old = np.array(y_new), np.array(y_old)
    
    # 2. Train base models on old data
    #base_models = ['L1-LR', 'L2-LR', 'DT', 'RF', 'LGBM']
    base_models = ['DT', 'RF', 'LGBM']
    base_models_dict = train_old_model(dataset, X_new, X_old, y_new, y_old)
    
    # 3. Initialize metrics dictionaries for both base and corrected models
    metrics = {
        'Accuracy': {model: [] for model in base_models + [f'{m}_LCT' for m in base_models]},
        'Precision': {model: [] for model in base_models + [f'{m}_LCT' for m in base_models]},
        'Recall': {model: [] for model in base_models + [f'{m}_LCT' for m in base_models]},
        'F1': {model: [] for model in base_models + [f'{m}_LCT' for m in base_models]}
    }
    
    # 4. Perform 5-fold cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_new, y_new)):
        print(f"Processing fold {fold + 1}/5")
        
        X_train, X_test = X_new[train_idx], X_new[test_idx]
        y_train, y_test = y_new[train_idx], y_new[test_idx]
        
        # Train LCT-corrected models on fold training data
        fold_corrected = train_new_models(dataset, X_train, y_train, base_models_dict, LocalCorrectionTree)
        
        # Evaluate both base and corrected models
        for model in base_models:
            # Evaluate base model
            if model == 'DNN':
                base_preds = base_models_dict[model].predict(X_test)
                final_preds_base = base_preds
            else:
                base_probs = base_models_dict[model].predict_proba(X_test)
                final_preds_base = np.argmax(base_probs, axis=1)
            
            # Calculate base model metrics
            metrics['Accuracy'][model].append(accuracy_score(y_test, final_preds_base))
            metrics['Precision'][model].append(precision_score(y_test, final_preds_base, average='macro'))
            metrics['Recall'][model].append(recall_score(y_test, final_preds_base, average='macro'))
            metrics['F1'][model].append(f1_score(y_test, final_preds_base, average='macro'))
            
            # Evaluate LCT-corrected model
            lct_model = fold_corrected[f'{model}_LCT']
            corrections = lct_model.predict(X_test)
            corrected_probs = base_probs + corrections
            final_preds_lct = np.argmax(corrected_probs, axis=1)
            
            # Calculate LCT model metrics
            metrics['Accuracy'][f'{model}_LCT'].append(accuracy_score(y_test, final_preds_lct))
            metrics['Precision'][f'{model}_LCT'].append(precision_score(y_test, final_preds_lct, average='macro'))
            metrics['Recall'][f'{model}_LCT'].append(recall_score(y_test, final_preds_lct, average='macro'))
            metrics['F1'][f'{model}_LCT'].append(f1_score(y_test, final_preds_lct, average='macro'))
    
    # 5. Prepare metrics for comparison plot
    plot_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    all_models = base_models + [f'{m}_LCT' for m in base_models]
    for metric in plot_metrics:
        for model in all_models:
            plot_metrics[metric].append(metrics[metric][model])
    
    # 6. Create and save comparison plot
    fig = plot_metrics_comparison(dataset, plot_metrics, all_models)
    plt.savefig(f'{dataset}_combined_comparison.png', bbox_inches='tight', dpi=400)
    plt.close()
    
    # 7. Print numerical results
    print(f"\nCombined Model Results for {dataset}:")
    for metric in plot_metrics:
        print(f"\n{metric}:")
        for name in all_models:
            mean_val = np.mean(metrics[metric][name])
            std_val = np.std(metrics[metric][name])
            print(f"{name}: {mean_val:.4f} ± {std_val:.4f}")

print("\nDone evaluating models.")


Processing Adult dataset


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternati

[LightGBM] [Info] Number of positive: 2326, number of negative: 8527
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001603 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 674
[LightGBM] [Info] Number of data points in the train set: 10853, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 2326, number of negative: 8527
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001716 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 664
[LightGBM] [Info] Number of data points in the train set: 10853, number of used features: 78
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


...

/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-04 19:45:27,997] Trial 99 finished with value: 0.7031056866631631 and parameters: {'learning_rate': 0.153900360918319, 'min_split_gain': 0.017505282289823784, 'reg_alpha': 0.031100706989262913, 'reg_lambda': 0.0017828966750929609}. Best is trial 36 with value: 0.7100064700948808.
/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 4352, number of negative: 11929
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 744
[LightGBM] [Info] Number of data points in the train set: 16281, number of used features: 87
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.267305 -> initscore=-1.008337
[LightGBM] [Info] Start training from score -1.008337


/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-02-04 19:45:28,523] A new study created in memory with name: no-name-79edd29e-0eaa-4301-9a58-70ec3493eb84


Processing fold 1/5


[I 2025-02-04 19:50:35,495] Trial 0 finished with value: -0.8458998771498771 and parameters: {'lambda_reg': 0.09508069760825165}. Best is trial 0 with value: -0.8458998771498771.
[I 2025-02-04 19:55:12,070] Trial 1 finished with value: -0.8467444717444718 and parameters: {'lambda_reg': 0.284301540813457}. Best is trial 1 with value: -0.8467444717444718.
[I 2025-02-04 19:59:56,879] Trial 2 finished with value: -0.8441339066339066 and parameters: {'lambda_reg': 0.48780444915394205}. Best is trial 1 with value: -0.8467444717444718.
[I 2025-02-04 20:05:04,026] Trial 3 finished with value: -0.8466676904176904 and parameters: {'lambda_reg': 0.15095587273981192}. Best is trial 1 with value: -0.8467444717444718.
[I 2025-02-04 20:09:51,033] Trial 4 finished with value: -0.8457463144963145 and parameters: {'lambda_reg': 0.22917617119891798}. Best is trial 1 with value: -0.8467444717444718.
[I 2025-02-04 20:14:56,372] Trial 5 finished with value: -0.8468980343980343 and parameters: {'lambda_reg':

In [ ]:
'''datasets = ['Adult']
for dataset in datasets:
    print(f"\nProcessing {dataset} dataset")
    
    # Define base and enhanced models
    base_models = ['L1-LR']# 'L2-LR', 'DT', 'RF', 'LGBM']
    enhanced_models = ['L1-LR+']#, 'L2-LR+', 'DT+', 'Ours']
                       #, 'RF+', 'LGBM+', 'LGBM+C', 'Ours']
    all_models = base_models + enhanced_models
    
    # 1. Load data
    if dataset == '7-point':
        X_old, y_old, X_new, y_new, X_clinical = load_prep_data(dataset)
    else:
        X_old, y_old, X_new, y_new, feature_names, x = load_prep_data(dataset)
    
    X_new, X_old = np.array(X_new[:100]), np.array(X_old[:100])
    y_new, y_old = np.array(y_new[:100]), np.array(y_old[:100])
    
    # 2. Train base models on old data
    base_models_dict = train_old_model(dataset, X_new, X_old, y_new, y_old)
    
    # 3. Train enhanced models on new data
    enhanced_models_dict = train_new_models(dataset, X_new, y_new, base_models_dict, LocalCorrectionTree)

    # 4. Perform 5-fold cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_metrics = {
        'Accuracy': {model: [] for model in base_models + list(enhanced_models_dict.keys())},
        'Precision': {model: [] for model in base_models + list(enhanced_models_dict.keys())},
        'Recall': {model: [] for model in base_models + list(enhanced_models_dict.keys())},
        'F1': {model: [] for model in base_models + list(enhanced_models_dict.keys())}
    }
        
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_new, y_new)):
        print(f"Processing fold {fold + 1}/5")
    
        X_train, X_test = X_new[train_idx], X_new[test_idx]
        y_train, y_test = y_new[train_idx], y_new[test_idx]
        
        # Evaluate each base model individually
        for model_name in base_models:
            fold_metrics = evaluate_old_model(dataset, X_test, y_test, base_models_dict[model_name])
            
            for metric in all_metrics:
                all_metrics[metric][model_name].extend(fold_metrics[metric])
        
        # Evaluate each enhanced model individually
        for model_name, model in enhanced_models_dict.items():
            old_model_scores = base_models_dict[model[:-4]].predict_proba(X_train)
            model.fit(X_train, y_train, old_model_scores)
            fold_metrics = evaluate_new_models(dataset, {model_name: model}, X_test, y_test, base_models_dict)
            
            for metric in all_metrics:
                all_metrics[metric][model_name].extend(fold_metrics[metric])

# 5. Prepare metrics for plotting
plot_metrics = {
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1': []
}

for metric in plot_metrics:
    for model in base_models + list(enhanced_models_dict.keys()):
        plot_metrics[metric].append(all_metrics[metric][model])

# 6. Create and save plot
fig = plot_metrics_comparison(dataset, plot_metrics, base_models + list(enhanced_models_dict.keys()))
plt.savefig(f'{dataset}_all_models_comparison.png', bbox_inches='tight', dpi=400)
plt.close()

# 7. Print numerical results
print(f"\nAll Model Results for {dataset}:")
for metric in plot_metrics:
    print(f"\n{metric}:")
    for name, values in zip(base_models + list(enhanced_models_dict.keys()), plot_metrics[metric]):
        mean_val = np.mean(values)
        std_val = np.std(values)
        print(f"{name}: {mean_val:.4f} ± {std_val:.4f}")

# 8. Print final Local Correction Tree structure
for model_name, model in enhanced_models_dict.items():
    if model_name.endswith('+'):
        print("\nFinal Local Correction Tree Structure:")
        model.simplify()
        print_tree(model, feature_names=feature_names)

print("\nDone evaluating all models.")'''

In [ ]:
'''datasets = ['Adult', 'Bank']
for dataset in datasets:
    print(f"\nProcessing {dataset} dataset")
    
    # Define only base models
    base_models = ['L1-LR', 'L2-LR', 'DT', 'RF', 'LGBM']
    
    # 1. Load data
    if dataset == '7-point':
        X_old, y_old, X_new, y_new, X_clinical = load_prep_data(dataset)
    else:
        X_old, y_old, X_new, y_new, feature_names, x = load_prep_data(dataset)
    
    X_new, X_old = np.array(X_new), np.array(X_old)
    y_new, y_old = np.array(y_new), np.array(y_old)
    
    # 2. Train base models on old data
    base_models_dict = train_old_model(dataset, X_new, X_old, y_new, y_old)
    
    # 3. Perform 5-fold cross validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_metrics = {
        'Accuracy': {model: [] for model in base_models},
        'Precision': {model: [] for model in base_models},
        'Recall': {model: [] for model in base_models},
        'F1': {model: [] for model in base_models}
    }
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_old, y_old)):
        print(f"Processing fold {fold + 1}/5")
        
        X_train, X_test = X_old[train_idx], X_old[test_idx]
        y_train, y_test = y_old[train_idx], y_old[test_idx]
        
        # Evaluate each base model individually
        for model_name in base_models:
            fold_metrics = evaluate_old_model(dataset, X_test, y_test, base_models_dict[model_name])
            
            for metric in all_metrics:
                all_metrics[metric][model_name].extend(fold_metrics[metric])
    
    # 4. Prepare metrics for plotting
    plot_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': []
    }
    
    for metric in plot_metrics:
        for model in base_models:
            plot_metrics[metric].append(all_metrics[metric][model])
    
    # 5. Create and save plot
    fig = plot_metrics_comparison(dataset, plot_metrics, base_models)
    plt.savefig(f'{dataset}_base_models_comparison.png', bbox_inches='tight', dpi=400)
    plt.close()
    
    # 6. Print numerical results
    print(f"\nBase Model Results for {dataset}:")
    for metric in plot_metrics:
        print(f"\n{metric}:")
        for name, values in zip(base_models, plot_metrics[metric]):
            mean_val = np.mean(values)
            std_val = np.std(values)
            print(f"{name}: {mean_val:.4f} ± {std_val:.4f}")

print("\nDone evaluating base models.")'''